# Part 1: ETL

This notebook collects the data using API connection and loads it into a table in SQL Server.

In [1]:
# importing libraries
%pip install alpha_vantage pyodbc

import requests
import pandas as pd
import pyodbc
import warnings

warnings.filterwarnings('ignore')

Note: you may need to restart the kernel to use updated packages.


In [2]:
'''
In this cell, two functions are created. The first one collects data of a single stock in the Alpha Vantage website using an API key,
and the second one iterates the given tickers making a request fo each one and adding the data into the DataFrame.
'''

# creating the API request function
def api_request(ticker):
    API_KEY = 'INSERT_YOUR_API_KEY_HERE'
    BASE_URL = 'https://www.alphavantage.co/query'

    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': ticker,
        'apikey': API_KEY,
        'datatype': 'json'
    }

    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        data = response.json()
        time_series = data.get('Time Series (Daily)', {})

        if time_series:
            df_stock_data = pd.DataFrame.from_dict(time_series, orient='index')
            df_stock_data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
            df_stock_data.index.name = 'Date'
            df_stock_data.reset_index(inplace=True)

            print(f'Data of {ticker} successfully collected.')
            return df_stock_data   # if the data is successfully collected, the functions returns a DataFrame, otherwise it returns errors messages and an empty DataFrame, as seen below
        
        else:
            print(f"Error: The API didn't return any data for {ticker} symbol.")
            return pd.DataFrame()
        
    else:
        print('An error occurred while accessing the API:', response.status_code, response.text)
        return pd.DataFrame()

# creating the loop for multiple inputs
def stock_prices_request(tickers):
    df_stock_prices_requested = pd.DataFrame()   # creates an empty DataFrame

    if not isinstance(tickers, (list, tuple)):   # verifies whether the input is a list or a tuple and, if it isn't, turns it into a list 
        tickers = [tickers]
    
    for ticker in tickers:
        df_ticker_data = api_request(ticker)   # iterates on each ticker

        if not df_ticker_data.empty:
            df_ticker_data['Ticker'] = ticker
            
            df_stock_prices_requested = pd.concat(
                [df_stock_prices_requested, df_ticker_data],   # adds the data into the initial DataFrame
                axis=0,
                ignore_index=True
            )

    return df_stock_prices_requested   # returns a unified DataFrame

In [ ]:
'''As an example, four enterprise stocks were chosen to test the pipeline.'''

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']   # Apple, Microsoft, Alphabet and Amazon
df_stock_prices = stock_prices_request(tickers)
df_stock_prices

Data of AAPL successfully collected.
Data of MSFT successfully collected.
Data of GOOGL successfully collected.
Data of AMZN successfully collected.


,Date,Open,High,Low,Close,Volume,Ticker
0,2025-01-15,234.6350,238.9600,234.4300,237.8700,39590680,AAPL
1,2025-01-14,234.7500,236.1200,232.4720,233.2800,39435294,AAPL
2,2025-01-13,233.5300,234.6700,229.7200,234.4000,49630725,AAPL
3,2025-01-10,240.0100,240.1600,233.0000,236.8500,61710856,AAPL
4,2025-01-08,241.9200,243.7123,240.0500,242.7000,37628940,AAPL
...,...,...,...,...,...,...,...
395,2024-08-28,173.6900,173.6900,168.9200,170.8000,29045025,AMZN
396,2024-08-27,174.1500,174.8900,172.2500,173.1200,29841979,AMZN
397,2024-08-26,176.7000,177.4682,174.3000,175.5000,22366236,AMZN
398,2024-08-23,177.3400,178.9699,175.2400,177.0400,29150091,AMZN


The table above shows the extracted data gathered into one DataFrame. The next step is to load those into a table in SQL Server.

In [4]:
# connecting to the SQL Server
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=INSERT_YOUR_SERVER_HERE;'
    'Database=StockData;'   # a database was previously created in SQL Server
    'Trusted_Connection=yes'
)

cursor = conn.cursor()

# creating the table that will receive the stock prices data
create_table_query = '''
CREATE TABLE StockPrices (
    [Date] DATE,
    [Open] FLOAT,
    [High] FLOAT,
    [Low] FLOAT,
    [Close] FLOAT,
    [Volume] BIGINT,
    [Ticker] NVARCHAR(10)
)
'''
cursor.execute(create_table_query)
conn.commit()

In [5]:
# iterating the DataFrame and inserting data into the SQL Server table
for index, row in df_stock_prices.iterrows():
    cursor.execute('''
    INSERT INTO StockPrices ([Date], [Open], [High], [Low], [Close], [Volume], [Ticker])
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''',
    row['Date'], row['Open'], row['High'], row['Low'], row['Close'], row['Volume'], row['Ticker']
    )
conn.commit()

print('Data were successfully inserted in SQL Server.')

conn.close()

Data were successfully inserted in SQL Server.
